# 🎭 AI Dad Joke Generator

Generate dad jokes with titles and cartoon illustrations using OpenAI API!

**What this does:**
- Creates a funny dad joke based on any subject
- Generates a catchy newspaper headline
- Makes a classical cartoon illustration

**Cost:** ~$0.05 per joke (GPT-4 + DALL-E 3)

## Step 1: Install Packages

In [ ]:
!pip install -q gradio openai pillow requests

## Step 2: Set Your OpenAI API Key

Get your key from: https://platform.openai.com/api-keys

In [ ]:
import os
from getpass import getpass

# Enter your API key (it won't be displayed)
api_key = getpass("Enter your OpenAI API Key: ")
os.environ['OPENAI_API_KEY'] = api_key
print("✅ API Key set!")

## Step 3: Import and Setup

In [ ]:
import gradio as gr
import openai
import requests
from io import BytesIO
from PIL import Image

# Initialize OpenAI client
client = openai.OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

## Step 4: Define Functions

In [ ]:
def generate_dad_joke(subject):
    """Generate a dad joke about the subject"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": "You are a dad joke expert. Create funny, wholesome dad jokes with clever wordplay and groan-worthy punchlines."
            },
            {
                "role": "user",
                "content": f"Create a short dad joke about: {subject}. Format: Setup: [setup] Punchline: [punchline]"
            }
        ],
        max_tokens=150,
        temperature=0.9
    )
    return response.choices[0].message.content.strip()


def generate_title(joke):
    """Generate a newspaper-style title"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {
                "role": "system",
                "content": "Create short, punchy, newspaper-style headlines (5-8 words max)."
            },
            {
                "role": "user",
                "content": f"Create a funny headline for this joke:\n{joke}\n\nReturn only the headline."
            }
        ],
        max_tokens=50,
        temperature=0.8
    )
    return response.choices[0].message.content.strip().strip('"')


def generate_cartoon(joke):
    """Generate a cartoon illustration"""
    prompt = f"""Classical black and white newspaper cartoon, 1950s style.
    Simple line art with cross-hatching. Family-friendly.
    Depicts: {joke}
    Style: Single panel comic strip, clean lines."""
    
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="standard",
        n=1
    )
    
    # Download the image
    image_url = response.data[0].url
    image_response = requests.get(image_url)
    image = Image.open(BytesIO(image_response.content))
    return image


def create_dad_joke_package(subject):
    """Main function - generates everything"""
    if not subject or subject.strip() == "":
        return "Please enter a subject!", "", None
    
    print(f"🎭 Generating dad joke about '{subject}'...")
    
    # Generate joke
    joke = generate_dad_joke(subject)
    print("😂 Joke created!")
    
    # Generate title
    title = generate_title(joke)
    print("📰 Title created!")
    
    # Generate cartoon
    image = generate_cartoon(joke)
    print("🎨 Cartoon created!")
    
    return joke, title, image

## Step 5: Create and Launch the Interface

In [ ]:
# Create the interface
with gr.Blocks(title="Dad Joke Generator", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🎭 AI Dad Joke Generator
        Enter any subject and get a dad joke with a title and cartoon!
        """
    )
    
    with gr.Row():
        with gr.Column(scale=1):
            subject_input = gr.Textbox(
                label="Subject or Theme",
                placeholder="e.g., coffee, cats, programming...",
                lines=2
            )
            generate_btn = gr.Button("Generate Dad Joke! 😄", variant="primary", size="lg")
            
            gr.Examples(
                examples=[["coffee"], ["cats"], ["computers"], ["pizza"], ["ocean"]],
                inputs=subject_input
            )
        
        with gr.Column(scale=2):
            title_output = gr.Textbox(label="📰 Headline", lines=2, interactive=False)
            joke_output = gr.Textbox(label="😂 The Dad Joke", lines=4, interactive=False)
            image_output = gr.Image(label="🎨 Cartoon Illustration", type="pil")
    
    # Connect button to function
    generate_btn.click(
        fn=create_dad_joke_package,
        inputs=subject_input,
        outputs=[joke_output, title_output, image_output]
    )

# Launch the app
demo.launch(share=True, inline=True)

---

## 🎉 That's it!

The interface should now be running above. Enter a subject and click "Generate Dad Joke!"

**Tips:**
- Try simple subjects like "dogs", "coffee", "coding"
- Generation takes 15-20 seconds
- Each joke costs about $0.05

**To use cheaper GPT-3.5-Turbo:** Change `model="gpt-4"` to `model="gpt-3.5-turbo"` in the functions above.